In [1]:
load_ext autoreload

In [2]:
autoreload 2

In [4]:
!ln -s /home/app/src/main.py main.py

In [5]:
from main import get_players_df

In [7]:
from datetime import datetime
date_str = datetime.today().strftime('%Y-%m-%d')

In [9]:
players_df = await get_players_df(date_str)

In [11]:
players_df

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type
0,NaN,None,37605,0,0,0,0,0,3,0.0,...,190.0,99.8,250,107,34,28,173,94,134,71
1,NaN,None,39476,0,0,0,0,0,2,0.0,...,110.0,58.5,142,53,290,105,220,64,230,74
2,NaN,None,41270,0,0,0,0,0,2,0.0,...,211.0,102.1,48,18,219,61,159,32,129,35
3,NaN,None,54694,0,0,0,0,0,3,0.0,...,1369.0,285.2,8,4,53,41,9,3,10,6
4,NaN,None,58822,0,0,0,0,0,2,0.0,...,118.0,68.7,180,68,157,34,219,63,199,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,NaN,None,216208,0,0,0,0,0,1,0.0,...,0.0,0.0,425,43,416,34,406,31,425,43
472,NaN,None,222531,0,0,0,0,0,3,0.0,...,12.0,6.3,362,150,282,141,314,146,345,146
473,NaN,None,247632,0,0,0,0,0,3,0.0,...,260.0,71.8,244,103,153,100,133,78,186,94
474,NaN,None,428610,0,0,0,0,0,3,0.0,...,0.0,1.3,367,154,326,149,412,164,367,154
